# Extract and Clean Data from all_matches.csv

Primero creamos un Spark Context

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import  (StructType, StructField, DateType, BooleanType, DoubleType, IntegerType, StringType, TimestampType)
from pyspark.sql.functions import col, udf
import os

spark = SparkSession.builder.master("local[1]").appName("tenis-matches").getOrCreate()

In [2]:
raw_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "raw", "all_matches.csv")
matches = spark.read.csv("file:///" + raw_file_path, header = True)

In [3]:
print("There are {} matches.".format(matches.count()))

There are 4295827 matches.


### Eliminaremos algunas columnas del dataset que no nos sirven

In [4]:
matches.printSchema()

root
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- court_surface: string (nullable = true)
 |-- prize_money: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- year: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- opponent_id: string (nullable = true)
 |-- opponent_name: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- round: string (nullable = true)
 |-- num_sets: string (nullable = true)
 |-- sets_won: string (nullable = true)
 |-- games_won: string (nullable = true)
 |-- games_against: string (nullable = true)
 |-- tiebreaks_won: string (nullable = true)
 |-- tiebreaks_total: string (nullable = true)
 |-- serve_rating: string (nullable = true)
 |-- aces: string (nullable = true)
 |-- double_faults: string (nullable = true)
 |-- first_serve_made: string (nullable = true)
 |-- first_serve_attempted:

Eliminaremos las siguientes columnas que no serán útiles para nuestro análisis:
- end_date
- location
- prize_money
- currency
- round
- serve_rating
- aces
- double_faults
- first_serve_made
- first_serve_attempted
- first_serve_points_made
- first_serve_points_attempted
- second_serve_points_made
- second_serve_points_attempted
- break_points_saved
- break_points_against
- service_games_won
- return_rating
- first_serve_return_points_made
- first_serve_return_points_attempted
- second_serve_return_points_made
- second_serve_return_points_attempted
- break_points_made
- break_points_attempted
- return_games_played
- service_points_won
- service_points_attempted
- return_points_won
- return_points_attempted
- total_points_won
- total_points
- duration
- seed
- masters
- round_num
- nation

In [5]:
matches = matches.drop("end_date")\
                .drop("location")\
                .drop("prize_money")\
                .drop("currency")\
                .drop("round")\
                .drop("serve_rating")\
                .drop("aces")\
                .drop("double_faults")\
                .drop("first_serve_made")\
                .drop("first_serve_attempted")\
                .drop("first_serve_points_made")\
                .drop("first_serve_points_attempted")\
                .drop("second_serve_points_made")\
                .drop("second_serve_points_attempted")\
                .drop("break_points_saved")\
                .drop("break_points_against")\
                .drop("service_games_won")\
                .drop("return_rating")\
                .drop("first_serve_return_points_made")\
                .drop("first_serve_return_points_attempted")\
                .drop("second_serve_return_points_made")\
                .drop("second_serve_return_points_attempted")\
                .drop("break_points_made")\
                .drop("break_points_attempted")\
                .drop("return_games_played")\
                .drop("service_points_won")\
                .drop("service_points_attempted")\
                .drop("return_points_won")\
                .drop("return_points_attempted")\
                .drop("total_points_won")\
                .drop("total_points")\
                .drop("duration")\
                .drop("seed")\
                .drop("masters")\
                .drop("round_num")\
                .drop("nation")

In [6]:
matches.printSchema()

root
 |-- start_date: string (nullable = true)
 |-- court_surface: string (nullable = true)
 |-- year: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- opponent_id: string (nullable = true)
 |-- opponent_name: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- num_sets: string (nullable = true)
 |-- sets_won: string (nullable = true)
 |-- games_won: string (nullable = true)
 |-- games_against: string (nullable = true)
 |-- tiebreaks_won: string (nullable = true)
 |-- tiebreaks_total: string (nullable = true)
 |-- player_victory: string (nullable = true)
 |-- retirement: string (nullable = true)
 |-- won_first_set: string (nullable = true)
 |-- doubles: string (nullable = true)



Eliminaremos registros con muchos valores nulos

In [7]:
# define functions to iterate in a generic way in dataframe

def print_null_value_count(df, column_name):
    null_values_count = df.where(df[column_name].isNull()).count()
    print("Cantidad de valores nulos para {0} : {1}.".format(column_name, null_values_count))
    
def get_null_values_in_dataframe(df):
    for column in df.columns:
        print_null_value_count(df, column)

In [8]:
get_null_values_in_dataframe(matches)

KeyboardInterrupt: 

En base a los datos obtenidos, es que se decide eliminar las siguientes columnas debido a que muchos valores son nulos, y además esta será obtenida de otro dataset:
- player_name
- opponent_name

In [8]:
matches = matches.drop("player_name").drop("opponent_name")

Luego, eliminaremos los datos donde tengan alguna propiedad nula.

In [9]:
matches = matches.na.drop()

Comprobamos no existan datos con propiedades nulas

In [ ]:
get_null_values_in_dataframe(matches)

In [12]:
print("Nueva cantidad de partidos : {0}".format(matches.select("*").count()))

Nueva cantidad de partidos : 4213346


### Modificaremos los tipos de algunos datos para que sean mas amigables luego

In [10]:
matches.printSchema()

root
 |-- start_date: string (nullable = true)
 |-- court_surface: string (nullable = true)
 |-- year: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- opponent_id: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- num_sets: string (nullable = true)
 |-- sets_won: string (nullable = true)
 |-- games_won: string (nullable = true)
 |-- games_against: string (nullable = true)
 |-- tiebreaks_won: string (nullable = true)
 |-- tiebreaks_total: string (nullable = true)
 |-- player_victory: string (nullable = true)
 |-- retirement: string (nullable = true)
 |-- won_first_set: string (nullable = true)
 |-- doubles: string (nullable = true)



Observaremos los tipos que deberían ser booleanos

In [13]:
boolean_columns = ["player_victory", "retirement", "won_first_set", "doubles"]
matches.select(boolean_columns).show(5)

+--------------+----------+-------------+-------+
|player_victory|retirement|won_first_set|doubles|
+--------------+----------+-------------+-------+
|             f|         f|            f|      f|
|             t|         f|            t|      f|
|             f|         f|            f|      f|
|             f|         f|            t|      t|
|             t|         f|            t|      f|
+--------------+----------+-------------+-------+
only showing top 5 rows



Pondremos valores de True y False, en lugar de f y t.

In [14]:
from pyspark.sql.functions import col,udf

boolParse = udf(lambda x: True if x == "t" else False, BooleanType())

def parse_boolean_columns(df, columns):
    for column in columns:
        df = df.withColumn(column, boolParse(col(column)))
    return df

In [15]:
matches = parse_boolean_columns(matches, boolean_columns)

In [16]:
matches.select(boolean_columns).show(5)

+--------------+----------+-------------+-------+
|player_victory|retirement|won_first_set|doubles|
+--------------+----------+-------------+-------+
|         false|     false|        false|  false|
|          true|     false|         true|  false|
|         false|     false|        false|  false|
|         false|     false|         true|   true|
|          true|     false|         true|  false|
+--------------+----------+-------------+-------+
only showing top 5 rows



Observaremos los tipos que deberían ser numeros

In [17]:
numeric_columns = ["num_sets", "sets_won", "games_won", "games_against", "tiebreaks_won", "tiebreaks_total"]
matches.select(numeric_columns).show(5)

+--------+--------+---------+-------------+-------------+---------------+
|num_sets|sets_won|games_won|games_against|tiebreaks_won|tiebreaks_total|
+--------+--------+---------+-------------+-------------+---------------+
|       2|       0|        3|           12|            0|              0|
|       3|       2|       16|           12|            0|              0|
|       2|       0|        7|           12|            0|              0|
|       3|       1|       18|           22|            1|              1|
|       3|       2|       17|           15|            1|              1|
+--------+--------+---------+-------------+-------------+---------------+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import col,udf

def parse_numeric_columns(df, columns):
    for column in columns:
        df = df.withColumn(column, col(column).astype(IntegerType()))
    return df

In [19]:
matches = parse_numeric_columns(matches, numeric_columns)
matches.printSchema()

root
 |-- start_date: string (nullable = true)
 |-- court_surface: string (nullable = true)
 |-- year: string (nullable = true)
 |-- player_id: string (nullable = true)
 |-- opponent_id: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- num_sets: integer (nullable = true)
 |-- sets_won: integer (nullable = true)
 |-- games_won: integer (nullable = true)
 |-- games_against: integer (nullable = true)
 |-- tiebreaks_won: integer (nullable = true)
 |-- tiebreaks_total: integer (nullable = true)
 |-- player_victory: boolean (nullable = true)
 |-- retirement: boolean (nullable = true)
 |-- won_first_set: boolean (nullable = true)
 |-- doubles: boolean (nullable = true)



Eliminaremos strings mal formados

In [1]:
def removeCharacter(df, column, char):
    removeFn = udf(lambda x: x.replace(char, ''), StringType())
    return df.withColumn(column, removeFn(col(column)))

In [2]:
matches = removeCharacter(matches, "tournament", '"')
matches = removeCharacter(matches, "player_id", '"')
matches = removeCharacter(matches, "opponent_id", '"')

NameError: name 'matches' is not defined

In [ ]:
processed_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "processed", "all_matches.csv")
matches.write.format("csv").option("header", True).mode('overwrite').save("file:///" + processed_file_path)

## Creacion de archivo para ejecutar extraccion de datos

In [ ]:
get_processed_data_script_file = os.path.join(os.path.pardir, "process_all_matches.py")

In [43]:
%%writefile $get_processed_data_script_file
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import  (StructType, StructField, DateType, BooleanType, DoubleType, IntegerType, StringType, TimestampType)
from pyspark.sql.functions import col, udf
import os

def extract_data():
    spark = SparkSession.builder.master("local[1]").appName("tenis-matches").getOrCreate()
    
    raw_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "raw", "all_matches.csv")
    matches = spark.read.csv("file:///" + raw_file_path, header = True)
    
    matches = matches.drop("end_date")\
                .drop("location")\
                .drop("prize_money")\
                .drop("currency")\
                .drop("round")\
                .drop("serve_rating")\
                .drop("aces")\
                .drop("double_faults")\
                .drop("first_serve_made")\
                .drop("first_serve_attempted")\
                .drop("first_serve_points_made")\
                .drop("first_serve_points_attempted")\
                .drop("second_serve_points_made")\
                .drop("second_serve_points_attempted")\
                .drop("break_points_saved")\
                .drop("break_points_against")\
                .drop("service_games_won")\
                .drop("return_rating")\
                .drop("first_serve_return_points_made")\
                .drop("first_serve_return_points_attempted")\
                .drop("second_serve_return_points_made")\
                .drop("second_serve_return_points_attempted")\
                .drop("break_points_made")\
                .drop("break_points_attempted")\
                .drop("return_games_played")\
                .drop("service_points_won")\
                .drop("service_points_attempted")\
                .drop("return_points_won")\
                .drop("return_points_attempted")\
                .drop("total_points_won")\
                .drop("total_points")\
                .drop("duration")\
                .drop("seed")\
                .drop("masters")\
                .drop("round_num")\
                .drop("nation")\
                .drop("player_name")\
                .drop("opponent_name")
    
    matches = matches.na.drop()
        
    boolean_columns = ["player_victory", "retirement", "won_first_set", "doubles"]
    matches = parse_boolean_columns(matches, boolean_columns)
    
    numeric_columns = ["num_sets", "sets_won", "games_won", "games_against", "tiebreaks_won", "tiebreaks_total"]
    matches = parse_numeric_columns(matches, numeric_columns)
    
    matches = removeCharacter(matches, "tournament", '"')
    matches = removeCharacter(matches, "player_id", '"')
    matches = removeCharacter(matches, "opponent_id", '"')
    
    return matches

def parse_boolean_columns(df, columns):
    boolParse = udf(lambda x: True if x == "t" else False, BooleanType())
    for column in columns:
        df = df.withColumn(column, boolParse(col(column)))
    return df

def parse_numeric_columns(df, columns):
    for column in columns:
        df = df.withColumn(column, col(column).astype(IntegerType()))
    return df

def removeCharacter(df, column, char):
    removeFn = udf(lambda x: x.replace(char, ''), StringType())
    return df.withColumn(column, removeFn(col(column)))

if __name__ == '__main__':
    df = extract_data()
    processed_file_path = os.path.join(os.path.abspath(os.path.pardir), "dataset", "processed", "all_matches.csv")
    df.write.format("csv").option("header", True).mode('overwrite').save("file:///" + processed_file_path)

Overwriting $get_processed_data_script_file


In [44]:
!python $get_processed_data_script_file

The system cannot find the path specified.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2020-11-15 17:36:42,391 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2020-11-15 17:36:42,392 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2020-11-15 17:36:48,633 WARN util.Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.

[Stage 1:>                                                          (0 + 1) / 7]
[Stage 1:========>                                                  (1 + 1) / 7]
[Stage 1:================>                                          (2 + 1) / 7]
[Stage 1:=========================>                                 (3 + 1) / 7]
[Stage 1:=================================>                         (4 + 1) / 7]

SUCCESS: The process with PID 10400 (child process of PID 16656) has been terminated.
SUCCESS: The process with PID 16656 (child process of PID 7872) has been terminated.
SUCCESS: The process with PID 7872 (child process of PID 4780) has been terminated.
